# NYT Spelling Bee Solver

Enter the Spelling Bee letters when prompted. The first letter is the required letter and must appear in every answer. The solver uses `nytbee_dict.txt` as the word list and prints hints in a format similar to the NYT Spelling Bee hint page.


In [ ]:
from __future__ import annotations

from collections import Counter
from pathlib import Path


def load_words(path: Path) -> list[str]:
    if not path.exists():
        raise FileNotFoundError(f"Missing wordlist: {path}")
    words = [line.strip().lower() for line in path.read_text().splitlines()]
    return [word for word in words if word.isalpha()]


def normalize_letters(letters: str) -> str:
    cleaned = ''.join(char for char in letters.lower() if char.isalpha())
    if len(cleaned) < 2:
        raise ValueError("Please provide at least two letters.")

    seen = set()
    unique = []
    for char in cleaned:
        if char not in seen:
            unique.append(char)
            seen.add(char)
    return ''.join(unique)


def solve_spelling_bee(letters: str, wordlist_path: Path) -> tuple[list[str], list[str], str]:
    normalized = normalize_letters(letters)
    required = normalized[0]
    allowed = set(normalized)

    words = [
        word
        for word in load_words(wordlist_path)
        if len(word) >= 4 and required in word and set(word) <= allowed
    ]
    pangrams = [word for word in words if set(normalized) <= set(word)]
    return sorted(words), sorted(pangrams), normalized


def summarize_by_length(words: list[str]) -> dict[int, int]:
    return dict(sorted(Counter(len(word) for word in words).items()))


def summarize_two_letters(words: list[str]) -> dict[str, int]:
    pairs = [word[:2] for word in words if len(word) >= 2]
    return dict(sorted(Counter(pairs).items()))


def print_hint_page(words: list[str], pangrams: list[str], letters: str) -> None:
    print('NYT Spelling Bee Hint Page')
    print('-' * 34)
    print(f"Letters: {', '.join(letters)} (required: {letters[0]})")
    print(f"Total words: {len(words)}")
    print(f"Pangrams ({len(pangrams)}): {', '.join(pangrams) if pangrams else 'None'}")

    length_counts = summarize_by_length(words)
    if length_counts:
        print('\nWord counts by length')
        for length, count in length_counts.items():
            print(f"  {length} letters: {count}")

    two_letter_counts = summarize_two_letters(words)
    if two_letter_counts:
        print('\nTwo-letter list')
        items = [f"{pair.upper()} ({count})" for pair, count in two_letter_counts.items()]
        line = []
        for item in items:
            line.append(item)
            if len(line) == 8:
                print('  ' + '  '.join(line))
                line = []
        if line:
            print('  ' + '  '.join(line))

    print('\nAnswers')
    grouped: dict[int, list[str]] = {}
    for word in words:
        grouped.setdefault(len(word), []).append(word)

    for length in sorted(grouped):
        word_list = ', '.join(sorted(grouped[length]))
        print(f"{length} letters ({len(grouped[length])}): {word_list}")


In [ ]:
letters_input = input('Enter the Spelling Bee letters (required letter first): ')
words, pangrams, normalized_letters = solve_spelling_bee(letters_input, Path('nytbee_dict.txt'))
print_hint_page(words, pangrams, normalized_letters)
